<a href="https://colab.research.google.com/github/VishwajithPk/image-classification-/blob/main/img_classification_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import tensorflow as tf
import numpy as np
import keras
from keras import *
from google.colab import files
from google.colab import drive
import cv2
from google.colab.patches import cv2_imshow
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import keras.utils as image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [15]:
img_dir = '/content/drive/MyDrive'

In [13]:
tf.random.set_seed(42)
train_dataset = tf.keras.utils.image_dataset_from_directory(img_dir,
                                                     validation_split=0.2,
                                                     subset="training",
                                                     seed=123,image_size=(200, 200),
                                                     batch_size=32)
tf.random.set_seed(42)
validation_dataset = tf.keras.utils.image_dataset_from_directory(img_dir,
                                                     validation_split=0.2,
                                                     subset="validation",
                                                     seed=123,image_size=(200, 200),
                                                     batch_size=32)

Found 1512 files belonging to 4 classes.
Using 1210 files for training.
Found 1512 files belonging to 4 classes.
Using 302 files for validation.


In [16]:
for image_batch, labels_batch in train_dataset:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

AUTOTUNE = tf.data.AUTOTUNE
tf.random.set_seed(42)
train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
tf.random.set_seed(42)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

(32, 200, 200, 3)
(32,)


In [17]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_ds = train_dataset.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image))

0.004221672 0.98776007


In [18]:
tf.random.set_seed(42)

img_model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(200, 200, 3)),
    tf.keras.layers.Conv2D(16, 3,(2,2), activation='relu', input_shape=(200,200,3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(32, 3,(2,2), activation='relu', input_shape=(200,200,3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64,3,(2,2), activation='relu', input_shape=(200,200,3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(128, activation='relu'),

    tf.keras.layers.Dense(32, activation='relu'),

    tf.keras.layers.Dense(1, activation='sigmoid')

])

In [19]:
img_model.compile(loss='binary_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.00004),
                  metrics=['categorical_accuracy'])


In [20]:
img_model.fit(train_dataset, epochs=5, validation_data=validation_dataset)

Epoch 1/5
38/38 [==============================] - 10s 234ms/step - loss: 0.1181 - categorical_accuracy: 1.0000 - val_loss: -0.4301 - val_categorical_accuracy: 1.0000
Epoch 2/5
38/38 [==============================] - 6s 152ms/step - loss: -1.2143 - categorical_accuracy: 1.0000 - val_loss: -2.1953 - val_categorical_accuracy: 1.0000
Epoch 3/5
38/38 [==============================] - 7s 189ms/step - loss: -3.8278 - categorical_accuracy: 1.0000 - val_loss: -6.1036 - val_categorical_accuracy: 1.0000
Epoch 4/5
38/38 [==============================] - 6s 145ms/step - loss: -10.6258 - categorical_accuracy: 1.0000 - val_loss: -17.2383 - val_categorical_accuracy: 1.0000
Epoch 5/5
38/38 [==============================] - 8s 209ms/step - loss: -29.1826 - categorical_accuracy: 1.0000 - val_loss: -45.1489 - val_categorical_accuracy: 1.0000


In [21]:
i = cv2.imread('/content/cat.4003.jpg')
i = cv2.resize(i, (200,200))
ar = np.asarray(i)
ar = tf.convert_to_tensor(ar)
ar = np.expand_dims(ar, axis=0)
v = np.vstack([ar])
val = img_model.predict(v)
print(val)

1/1 [==============================] - 0s 95ms/step
[[1.]]
